In [1]:
import cv2
import cv2.cv2
import numpy as np

### 车辆统计项目

In [4]:
cap = cv2.VideoCapture('video.mp4')
while True:
    ret, frame = cap.read()
    if ret:
        cv2.imshow('video', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### 去背景（背景减除法）
背景减除(Background Subtraction)是许多基于计算机视觉的任务中的主要预处理步骤。
如果我们有完整的静止的背景帧，那么我们可以通过帧差法来计算像素差从而获取到前景对象。
但是在大多数情况下，我们可能没有这样的图像，所以我们需要从我们拥有的任何图像中提取背景。
当运动物体有阴影时，由于阴影也在移动，情况会变的变得更加复杂。为此引入了背景减除算法，通过这一方法我们能够从视频中分离出运动的物体前景，从而达到目标检测的目的。


如果您使用的是python3，请输入：

fgbg =cv2.createBackgroundSubtractorMOG2() #For python3

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

+ BackgroundSubtractorMOG
+ 这是一个以混合高斯模型为基础的前景/背景分割算法。它是 P.KadewTraKuPong和R.Bowden在2001年提出的。
+ 它使用K(K=3或5)个高斯分布混合对背景像素进行建模。使用这些颜色(在整个视频中)存在时间的长短作为混合的权重。背景的颜色一般持续的时间最长，而且更加静止。
+ 在编写代码时，我们需要使用函数:cv2.bgsegm.createBackgroundSubtractorMOG()创建一个背景对象。这个函数有些可选参数，比如要进行建模场景的时间长度，高斯混合成分的数量，阈值等。将他们全部设置为默认值。然后在整个视频中我们是需要使用backgroundsubtractor.apply() 就可以得到前景的掩模了.。移动的物体会被标记为白色,背景会被标记为黑色的


### 获取前景掩码

In [4]:
cap = cv2.VideoCapture(0)
#  pip install opencv-contrib-python
mog = cv2.createBackgroundSubtractorMOG2() #For python3
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # 前景的掩膜
    fgmask = mog.apply(frame)
    cv2.imshow('fgmask', fgmask)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### 形态学识别车辆

In [16]:
cap = cv2.VideoCapture('video.mp4')
#  pip install opencv-contrib-python
mog = cv2.createBackgroundSubtractorMOG2() #For python3
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # 前景的掩膜

    # cv2.imshow('fgmask', fgmask)
    # 思路：有噪声点，先去除
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(3,3),5)
    fgmask = mog.apply(blur)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    erode = cv2.erode(fgmask, kernel)
    dilate = cv2.dilate(erode, kernel, iterations=2)
    # 消除内部的小方块
    close = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, kernel)
    cv2.imshow('img', close)

    key = cv2.waitKey(1)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### 判断是否是车辆

In [59]:
cap = cv2.VideoCapture('video.mp4')
#  pip install opencv-contrib-python
mog = cv2.createBackgroundSubtractorMOG2() #For python3

cars = []
car_no = 0

while True:
    ret, frame = cap.read()
    fps = cap.get(cv2.CAP_PROP_FPS)

    if not ret:
        break
    # 前景的掩膜

    # cv2.imshow('fgmask', fgmask)
    # 思路：有噪声点，先去除
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(3,3),5)
    fgmask = mog.apply(blur)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    erode = cv2.erode(fgmask, kernel)
    dilate = cv2.dilate(erode, kernel, iterations=2)
    # 消除内部的小方块
    close = cv2.morphologyEx(dilate, cv2.MORPH_CLOSE, kernel)

    #### 识别车辆 ####
    contours, h = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    line_high = 600
    offset = 25
    def center(x, y, w, h):
        return int(x + w / 2), int(y + w / 2)
    cv2.line(frame, (100, line_high), (1180, line_high), (255,255,0),3)
    cv2.line(frame, (100, line_high+offset), (1180, line_high+offset), (0,255,0),1)
    cv2.line(frame, (100, line_high-offset), (1180, line_high-offset), (0,255,0),1)
    for contour in contours:
        # 左上角xy、w，h
        x, y, w, h = cv2.boundingRect(contour)
        x, y, w, h = int(x), int(y), int(w), int(h)
        # 过滤掉小框
        min_w = 90
        min_h = 80
        if w < min_w or h < min_h:
            break
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        # cv2.putText(frame,'yes',(x,y),cv2.FONT_HERSHEY_TRIPLEX, 1, (255,0,0))
        x, y = center(x,y,w,h)
        cars.append((x,y))
        cv2.circle(frame, (x, y), 5, (0,255,255), -1)
        # 判断车是否过线（上下两条）
        for (x,y) in cars:
            if y > (line_high - offset) or y < (line_high + offset):
                car_no += 1
                cars.remove((x,y))


    cv2.putText(frame,str(car_no),(100,100),cv2.FONT_HERSHEY_TRIPLEX, 1, (255,0,0))

    cv2.imshow('img', frame)

    key = cv2.waitKey(int(1000/fps))
    if key == ord('q'):
        break

cars.clear()
cap.release()
cv2.destroyAllWindows()